## Data Prerocessing

### Reading the dataset

Comma-separated values (CSV) files are ubiquitous for the storing of
tabular (spreadsheet-like) data. In them, each line corresponds to one
record and consists of several (comma-separated) fields, e.g., "Albert
Einstein,March 14 1879,Ulm,Federal polytechnic school,field of
gravitational physics". To demonstrate how to load CSV files with
``pandas``, we create a CSV file below ``../data/house_tiny.csv``. This
file represents a dataset of homes, where each row corresponds to a
distinct home and the columns correspond to the number of rooms
(``NumRooms``), the roof type (``RoofType``), and the price (``Price``)

In [3]:
import os

os.makedirs(os.path.join("..", "data"), exist_ok = True)
data_file = os.path.join("..", "data", "house_tiny.csv")
with open(data_file, "w") as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')


In [4]:
import pandas as pd 
data = pd.read_csv(data_file)
data

,NumRooms,RoofType,Price
0,NaN,NaN,127500
1,2.0,NaN,106000
2,4.0,Slate,178100
3,NaN,NaN,140000


### Data Preparation


In supervised learning, we train models to predict a designated *target*
value, given some set of *input* values. Our first step in processing
the dataset is to separate out columns corresponding to input versus
target values. We can select columns either by name or via
integer-location based indexing (``iloc``).

You might have noticed that ``pandas`` replaced all CSV entries with
value ``NA`` with a special ``NaN`` (*not a number*) value. This can
also happen whenever an entry is empty, e.g., "œ3,,,270000". These are
called *missing values* and they are the "bed bugs" of data science, a
persistent menace that you will confront throughout your career.
Depending upon the context, missing values might be handled either via
*imputation* or *deletion*. Imputation replaces missing values with
estimates of their values while deletion simply discards either those
rows or those columns that contain missing values.

Here are some common imputation heuristics. For categorical input
fields, we can treat ``NaN`` as a category. Since the ``RoofType``
column takes values ``Slate`` and ``NaN``, ``pandas`` can convert this
column into two columns ``RoofType_Slate`` and ``RoofType_nan``. A row
whose roof type is ``Slate`` will set values of ``RoofType_Slate`` and
``RoofType_nan`` to 1 and 0, respectively. The converse holds for a row
with a missing ``RoofType`` value.


In [5]:
inputs, targets = data.iloc[:, 0: 1+1], data.iloc[:, 2] #take input and target(the output) using slicing and iloc
inputs

,NumRooms,RoofType
0,NaN,NaN
1,2.0,NaN
2,4.0,Slate
3,NaN,NaN


In [6]:
targets

0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64

In [7]:
inputs = pd.get_dummies(inputs, dummy_na= True)
inputs

,NumRooms,RoofType_Slate,RoofType_nan
0,NaN,False,True
1,2.0,False,True
2,4.0,True,False
3,NaN,False,True


### get_dummies()
Convert categorical variable into dummy/indicator variables.
```bash
pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)
```
Each variable is converted in as many 0/1 variables as there are different values. Columns in the output are each named after a value; if the input is a DataFrame, the name of the original variable is prepended to the value.

#### Parameters

- **data** : array-like, Series, or DataFrame  
  Data of which to get dummy indicators.

- **prefix** : str, list of str, or dict of str, default `None`  
  String to append DataFrame column names.  
  - Pass a list with length equal to the number of columns when calling `get_dummies` on a DataFrame.  
  - Alternatively, `prefix` can be a dictionary mapping column names to prefixes.

- **prefix_sep** : str, default `'_'`  
  Separator/delimiter to use when appending prefix.  
  - Can also pass a list or dictionary as with `prefix`.

- **dummy_na** : bool, default `False`  
  - If `True`, add a column to indicate NaNs.  
  - If `False`, NaNs are ignored.

- **columns** : list-like, default `None`  
  Column names in the DataFrame to be encoded.  
  - If `None`, then all the columns with object, string, or category dtype will be converted.

- **sparse** : bool, default `False`  
  Whether the dummy-encoded columns should be backed by a `SparseArray` (`True`) or a regular NumPy array (`False`).

- **drop_first** : bool, default `False`  
  Whether to get `k-1` dummies out of `k` categorical levels by removing the first level.

- **dtype** : dtype, default `bool`  
  Data type for new columns. Only a single dtype is allowed.

---

#### Returns

- **DataFrame**  
  Dummy-coded data.  
  - If `data` contains other columns than the dummy-coded one(s), these will be prepended, unaltered, to the result.


For missing numerical values, one common heuristic is to replace the NaN entries with the mean value of the corresponding column.

In [8]:
inputs = inputs.fillna(inputs.mean())
inputs

,NumRooms,RoofType_Slate,RoofType_nan
0,3.0,False,True
1,2.0,False,True
2,4.0,True,False
3,3.0,False,True


### Conversion to the Tensor Format
Now all that entries `inputs` and `targets` are numberical, we can load them into a tensor

In [9]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))

X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

dataframe -> numpy -> tensor

### Exercises

Try loading datasets, e.g., Abalone from the UCI Machine Learning Repository and inspect their properties. What fraction of them has missing values? What fraction of the variables is numerical, categorical, or text?

In [14]:
data_path = os.path.join("..","data", "abalone", "abalone.data")
data_path

'..\\data\\abalone\\abalone.data'

In [20]:
head = ["sex", "length", "diameter", "height", "whole_weight", "suchked_weight", "viscera_weight", "shell_weight", "rings"]

In [22]:
data = pd.read_csv(data_path, names = head)
data

,sex,length,diameter,height,whole_weight,suchked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [31]:
data.isna().sum(axis=0) / len(data) * 100

sex               0.0
length            0.0
diameter          0.0
height            0.0
whole_weight      0.0
suchked_weight    0.0
viscera_weight    0.0
shell_weight      0.0
rings             0.0
dtype: float64

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sex             4177 non-null   object 
 1   length          4177 non-null   float64
 2   diameter        4177 non-null   float64
 3   height          4177 non-null   float64
 4   whole_weight    4177 non-null   float64
 5   suchked_weight  4177 non-null   float64
 6   viscera_weight  4177 non-null   float64
 7   shell_weight    4177 non-null   float64
 8   rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


Try indexing and selecting data columns by name rather than by column number. The pandas documentation on indexing has further details on how to do this.

In [ ]:
data["sex"][34]

'F'

In [36]:
data["length"][12]

0.49

In [38]:
data[["sex", "shell_weight"]][2:4]

,sex,shell_weight
2,F,0.210
3,M,0.155


How large a dataset do you think you could load this way? What might be the limitations? Hint: consider the time to read the data, representation, processing, and memory footprint. Try this out on your laptop. What happens if you try it out on a server?

In [39]:
import time

start = time.time()
df = pd.read_csv(data_path)
time.time() - start

0.005578517913818359